# Looking into privacy policies readability

website with good visualizations https://vpnoverview.com/research/most-difficult-to-read-privacy-policies/

In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
spreadsheet_key = '1-v7zfebjdHbfFHcoNftxW7bsW1xQ2zUNvjLmH6Q4OnU'
spreadsheet_gid = '0'
csv_link = 'https://docs.google.com/spreadsheets/d/'+spreadsheet_key+'/export?gid='+spreadsheet_gid+'&format=csv'
df = pd.read_csv(csv_link)
df.sort_values(by=['Words per Sentence'], ascending=False, inplace=True)
df

In [ ]:
#df.plot.bar(x='Website', y='Words per Sentence', rot=90)
px.bar(df, y='Website', x='Words per Sentence', title='Words per Sentence in Privacy Policies', orientation='h', height=1200)

In [ ]:
#df.iloc[0:19].plot.bar(x='Website', y='Words per Sentence', rot=90)
px.bar(df.head(20), y='Website', x='Words per Sentence', title='Words per Sentence in Privacy Policies', orientation='h', height=600)

In [ ]:
#df.iloc[82:102].plot.bar(x='Website', y='Words per Sentence', rot=90)
px.bar(df.iloc[82:102], y='Website', x='Words per Sentence', title='Words per Sentence in Privacy Policies', orientation='h', height=600)

In [ ]:
df1 = df.sort_values(by=['Word count'], ascending=False)
#df1.plot.bar(x='Website', y='Word count', rot=90)
px.bar(df1, y='Website', x='Word count', title='Word Counts in Privacy Policies', orientation='h', height=1200)

In [ ]:
#df1.iloc[0:19].plot.bar(x='Website', y='Word count', rot=90)
px.bar(df1.head(20), y='Website', x='Word count', title='Word Counts in Privacy Policies', orientation='h', height=600)

In [ ]:
#df1.iloc[82:102].plot.bar(x='Website', y='Word count', rot=90)
px.bar(df1.iloc[82:102], y='Website', x='Word count', title='Word Counts in Privacy Policies', orientation='h', height=600)

In [ ]:
#df.plot.scatter(x = 'Word count', y = 'Overall Readability\nscore')
px.scatter(df, x='Word count', y='Overall Readability\nscore', title='Readability Scores in Privacy Policies', trendline='ols', height=600)

In [ ]:
df['Word count'].corr(df['Overall Readability\nscore'])

In [ ]:
#df.plot.scatter(x = 'Words per Sentence', y = 'Overall Readability\nscore')
px.scatter(df, x='Words per Sentence', y='Overall Readability\nscore', title='Readability Scores in Privacy Policies', trendline='ols', height=600)

In [ ]:
df['Words per Sentence'].corr(df['Overall Readability\nscore'])

# Doing our own analysis of privacy policies

Here is a study from Cornell University that has curated and analyzed over a million privacy policies that span over two decades.

Github repo of privacy policies they have collected: https://github.com/citp/privacy-policy-historical 